# Visualization notebook

The goal of this notebook is to provide visualization of data using Kepler.gl


In [1]:
from keplergl import KeplerGl
import geopandas as gpd
import pandas as pd
import numpy as np

In [2]:
# --- Global variables

# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account
dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

In [3]:
def to_gdf(path):
    """
    Parameters: 
        path: path of the file to read as a geodataframe
        
    return:
        a GeoDataFrame (with Geopandas) corresponding to the file path
    """
    gdf = gpd.GeoDataFrame.from_file(path)
    gdf = gdf.to_crs('epsg:4326')
    return gdf

In [4]:
import ast
config_file = open('visualization_config.txt', 'r')
text = config_file.read()
dict_config = ast.literal_eval(text)
config_file.close()

## 1. Network data

## Project delimitation

In [37]:
project_del_path = dbx + "/Private Structured data collection/Manual-made dataset (do not touch)/Network/Map/Project Delimitation/Project_delimitation.shp"
project_del = to_gdf(project_del_path)

# tmp = project_del['geometry']
# tmp[0] = poly

# new_df = pd.DataFrame({'geometry': tmp})
# project_del.update(new_df)
# print(tmp)
# # project_del.iloc[0]['geometry'] = poly
# print(project_del)

# project_del.to_file(project_del_path)
print(project_del)
print(project_del[project_del.Type == "Box"])

fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = project_del[project_del.Type == "Box"], name="Project box delimitation")
fremont_map.add_data(data = project_del[project_del.Type == "Delimitation"], name="Project delimitation")
fremont_map


   OBJECTID  Shape_Leng  Shape_Area          Type  \
0         3    0.263470    0.004303           Box   
1         4    0.224795    0.002496  Delimitation   

                                            geometry  
0  POLYGON ((-121.95440 37.49070, -121.95440 37.5...  
1  POLYGON ((-121.94277 37.55273, -121.94100 37.5...  
   OBJECTID  Shape_Leng  Shape_Area Type  \
0         3     0.26347    0.004303  Box   

                                            geometry  
0  POLYGON ((-121.95440 37.49070, -121.95440 37.5...  
User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Project box delimitation': {'index': [0], 'columns': ['OBJECTID', 'Shape_Leng', 'Shape_Area', …

## Aimsun map

## To do Theo: get new network data from Aimsun and plot with new data when every works (centroids, centroid connections, etc.)

In [6]:
aimsun_path_regular = dbx + '/Private Structured data collection/Aimsun/Inputs/'
aimsun_path_complex = dbx + '/Private Structured data collection/Aimsun/Inputs_wk_26_OD_4_12_2020/'

aimsun_path = aimsun_path_complex

detectors = to_gdf(aimsun_path +'detectors.shp')
meterings = to_gdf(aimsun_path +'meterings.shp')
centroids = to_gdf(aimsun_path +'centroids.shp')
centroid_connections = to_gdf(aimsun_path +'centroid_connections.shp')
nodes = to_gdf(aimsun_path +'nodes.shp')
# polygons = to_gdf(aimsun_path +'polygons.shp')
sections = to_gdf(aimsun_path +'sections.shp')
sectionsGeo = to_gdf(aimsun_path +'sectionsGeo.shp')
turnings = to_gdf(aimsun_path +'turnings.shp')

#### The following cell is useful to better understand and render the network
## To do Theo: get the different screenshot of the section characteristics

In [7]:

# fremont_map = KeplerGl(height=600)

node_type = np.sort(nodes.nodetype.unique())
print("--------Nodes-------")
print()
print("Total number of nodes: " + str(nodes['id'].count()))
print("Nodes type values: " + str(node_type))
for i in node_type:
    print("Number of nodes with type " + str(i) + ": " + str(nodes[nodes['nodetype'] == i]['id'].count()))
    if False:
        fremont_map.add_data(data = nodes[nodes['nodetype'] == i], name="Nodes type " + str(i))

    
print()
print("------Sections------")

print("Total number of section: " + str(sections['id'].count()))
print()
section_type = np.sort(sections.rd_type.unique())
print("Road type values: " + str(section_type))
print("175 = Motorway, 177 = Primary, 179 = Residential, 180 = Secondary, 182.0 = Tertiary, 184.0 = Trunk, 185.0 = Unclassified")
for i in section_type:
    print("Number of sections with type " + str(i) + ": " + str(sections[sections['rd_type'] == i]['id'].count()))
    if False:
        fremont_map.add_data(data = sections[sections['rd_type'] == i], name="Sections type " + str(i))
    
print()
section_speed = np.sort(sections.speed.unique())
print("Speed values: " + str(section_speed))
for i in section_speed:
    print("Number of sections with speed " + str(i) + " (km/h): " + str(sections[sections['speed'] == i]['id'].count()))
    if False:
        fremont_map.add_data(data = sections[sections['speed'] == i], name="Sections speed " + str(i))

print()
section_cap = np.sort(sections.capacity.unique())
print("Capacity values: " + str(section_cap))
for i in section_cap:
    print("Number of sections with capacity " + str(i) + " (veh/h): " + str(sections[sections['capacity'] == i]['id'].count()))
    if False:
        fremont_map.add_data(data = sections[sections['capacity'] == i], name="Sections capacity " + str(i))
    
print()
section_func = np.sort(sections.func_class.unique())
print("Function class values: " + str(section_func))
for i in section_func:
    print("Number of sections with function class " + str(i) + ": " + str(sections[sections['func_class'] == i]['id'].count()))
    if False:
        fremont_map.add_data(data = sections[sections['func_class'] == i], name="Sections function class " + str(i))
    
# fremont_map

--------Nodes-------

Total number of nodes: 2640
Nodes type values: [0. 1. 2. 3.]
Number of nodes with type 0.0: 774
Number of nodes with type 1.0: 1632
Number of nodes with type 2.0: 44
Number of nodes with type 3.0: 190

------Sections------
Total number of section: 7320

Road type values: [175. 177. 179. 180. 182. 184. 185.]
175 = Motorway, 177 = Primary, 179 = Residential, 180 = Secondary, 182.0 = Tertiary, 184.0 = Trunk, 185.0 = Unclassified
Number of sections with type 175.0: 150
Number of sections with type 177.0: 439
Number of sections with type 179.0: 3368
Number of sections with type 180.0: 498
Number of sections with type 182.0: 286
Number of sections with type 184.0: 62
Number of sections with type 185.0: 2517

Speed values: [ 16.  40.  50.  56.  64.  72.  88.  90. 104. 120.]
Number of sections with speed 16.0 (km/h): 4
Number of sections with speed 40.0 (km/h): 44
Number of sections with speed 50.0 (km/h): 6717
Number of sections with speed 56.0 (km/h): 176
Number of sect

In [8]:
# centroid_config = centroid_map.config
# file.write(",'centroid_config': " + str(centroid_config))
# file.write("}")
# file.close()

In [9]:
fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = detectors, name="Detectors")
fremont_map.add_data(data = meterings, name="meterings")
fremont_map.add_data(data = nodes, name="nodes")
# fremont_map.add_data(data = polygons, name="polygons")
fremont_map.add_data(data = sections, name="sections")
fremont_map.add_data(data = sectionsGeo, name="sectionsGeo")
fremont_map.add_data(data = turnings, name="turnings")
fremont_map.add_data(data = centroids, name="centroids")
fremont_map.add_data(data = centroid_connections, name="centroid_connections")
fremont_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Detectors': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2…

## Aimsun centroids and centroids connections

In [10]:
centroid_map = KeplerGl(height=600, config_file=dict_config['centroid_config'])
centroid_map.add_data(data = centroids[centroids['name'].str.contains('ext')], name="External centroids")
centroid_map.add_data(data = centroids[centroids['name'].str.contains('int')], name="Internal centroids")
centroid_map.add_data(data = centroid_connections, name="Centroid connections")
# fremont_map.add_data(data = centroid_connections[centroid_connections['direction'] == 'from'], name="Outgoing centroid connections")
# fremont_map.add_data(data = centroid_connections[centroid_connections['direction'] == 'to'], name="Incoming centroid connections")
centroid_map.add_data(data = sections, name="sections")
centroid_map


User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'External centroids': {'index': [97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107], 'columns'…

## Aimsun road section

In [11]:
road_section_map = KeplerGl(height=600, config=dict_config['road_type_config'])

road_section_map.add_data(data = sections[sections['rd_type'] == 175.0], name="Motorway")
road_section_map.add_data(data = sections[sections['rd_type'] == 177.0], name="Primary")
road_section_map.add_data(data = sections[sections['rd_type'] == 179.0], name="Residential")
road_section_map.add_data(data = sections[sections['rd_type'] == 180.0], name="Secondary")
road_section_map.add_data(data = sections[sections['rd_type'] == 182.0], name="Tertiary")
road_section_map.add_data(data = sections[sections['rd_type'] == 184.0], name="Trunk")
road_section_map.add_data(data = sections[sections['rd_type'] == 185.0], name="Unclassified")

road_section_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'cmvn55k', 'type': …

In [12]:
# road_type_config = road_section_map.config
# file = open("visualization_config.txt", 'w')
# file.write("{'road_type_config': " + str(road_type_config))
# file.write("}")
# file.close()

## Traffic signals and stop signs

In [13]:
def to_gdf_csv(path):
# https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html#from-wkt-format
    df = pd.read_csv(path)
    gdf = gpd.GeoDataFrame(
        df, crs='epsg:4326', geometry=gpd.points_from_xy(df.x, df.y))
    return gdf

In [14]:
network_infra_path = data_path + "/Manual-made dataset (do not touch)/Network/Infrastructure/"

stop_signs = to_gdf_csv(network_infra_path + "Stop signs location/Stop_Signs.csv")
traffic_lights = to_gdf_csv(network_infra_path + "Traffic lights location/Traffic_Lights.csv")

In [15]:
fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = stop_signs, name="Stop signs")
fremont_map.add_data(data = traffic_lights, name="Traffic lights")
fremont_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Stop signs': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, …

In [16]:
print("Number of stop signs: " + str(stop_signs.__OBJECTID.count()))
print("Number of traffic lights: " + str(traffic_lights.__OBJECTID.count()))


Number of stop signs: 313
Number of traffic lights: 123


## 2. Demand data

## Transportation Analysis Zones

## To do Ayush: 
1. Use the external centroids (and not centroid of the external TAZs) to plot the desire lines. Put in red external demand. Put in blue internal demand. Maybe cluster some internal TAZs together to get a better rendering of the demand data. Set the width of the lines to be a function of the lines. If possible get some legend about how width is related to count.
2. Get the following information:
    - Between 2 and 8pm: YYY residential cars (YYY = sum of count over every row of internal_od when dt_15 is between 14:00 and 20:00)
    - Between 2 and 8pm: ZZZ traveler cars (ZZZ = sum of count over every row of external_od when dt_15 is between 14:00 and 20:00)


In [17]:
from shapely.geometry import Point, LineString

path_taz = data_path + "/Data processing/Auxiliary files/Demand/OD demand/TAZ"
internal_taz = to_gdf(path_taz + "/Internal_TAZ.shp")
external_taz = to_gdf(path_taz + "/External_TAZ.shp")

# Get gravity centers for all TAZs (internal and external)
centroid_gravity = {}
for i in range(len(internal_taz['geometry'])):
    centroid_gravity[internal_taz['CentroidID'][i]] = internal_taz['geometry'][i].centroid
for i in range(len(external_taz['geometry'])):
    centroid_gravity[external_taz['CentroidID'][i]] = external_taz['geometry'][i].centroid

od_demand_path = data_path + "/Data processing/Temporary exports to be copied to processed data/Demand/OD demand/" 
internal_od = pd.read_csv(od_demand_path + "Internal OD grouped by timestamp.csv")
external_od = pd.read_csv(od_demand_path + "External OD grouped by timestamp.csv")

# Remove centroids with no demand
external_od = external_od[external_od["counts"] != 0]
internal_od = internal_od[internal_od["counts"] != 0]
internal_od = internal_od[internal_od["CentroidID_O"] != "int_16"]
internal_od = internal_od[internal_od["CentroidID_D"] != "int_16"]
internal_od = internal_od[internal_od["CentroidID_O"] != "int_62"]
internal_od = internal_od[internal_od["CentroidID_D"] != "int_62"]
# Fix analysis timestep at 6 PM
internal_od_6_pm = internal_od[internal_od["dt_15"]=="18:00"]
external_od_6_pm = external_od[external_od["dt_15"]=="18:0"]

internal_od_6_pm = internal_od_6_pm.reset_index()
external_od_6_pm = external_od_6_pm.reset_index()

external_demand = gpd.GeoDataFrame(columns=['CentroidID_O', 'CentroidID_D', "counts", 'geometry'])
for i in range(len(external_od_6_pm['CentroidID_O'])):
    origin_id = external_od_6_pm['CentroidID_O'][i]
    dest_id = external_od_6_pm['CentroidID_D'][i]
    demand = external_od_6_pm['counts'][i]
    external_demand.loc[i] = [origin_id, dest_id, demand, LineString([centroid_gravity[origin_id], centroid_gravity[dest_id]])]

internal_demand = gpd.GeoDataFrame(columns=['CentroidID_O', 'CentroidID_D', "counts", 'geometry'])
internal_external_demand = gpd.GeoDataFrame(columns=['CentroidID_O', 'CentroidID_D', "counts", 'geometry'])
for i in range(len(internal_od_6_pm['CentroidID_O'])):
    origin_id = internal_od_6_pm['CentroidID_O'][i]
    dest_id = internal_od_6_pm['CentroidID_D'][i]
    demand = internal_od_6_pm['counts'][i]
    if "ext" not in origin_id and "ext" not in dest_id:
        internal_demand.loc[i] = [origin_id, dest_id, demand, LineString([centroid_gravity[origin_id], centroid_gravity[dest_id]])]
    else:
        internal_external_demand.loc[i] = [origin_id, dest_id, demand, LineString([centroid_gravity[origin_id], centroid_gravity[dest_id]])]

print("Number of Vehicles Analyzed at 6 PM (Internal):", internal_od_6_pm.counts.sum())
print("Number of Vehicles Analyzed at 6 PM (External):", external_od_6_pm.counts.sum())
print("Number of Vehicles Analyzed at 6 PM (Total):", internal_od_6_pm.counts.sum() + external_od_6_pm.counts.sum())

fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = external_demand, name="External/External Demand")
fremont_map.add_data(data = internal_demand, name="Internal/Internal Demand")
fremont_map.add_data(data = internal_external_demand, name="Internal/External Demand")
fremont_map.add_data(data = internal_taz, name="Internal TAZs")
fremont_map.add_data(data = external_taz, name="External TAZs")
fremont_map

'''
fremont_map.save_to_html(file_name=data_path+"/Data processing/Kepler maps/Demand/6pm_demand_map.html")
frm_config = fremont_map.config
frm_map = KeplerGl(height=600, config=frm_config)
frm_map.add_data(data = external_demand, name="External/External Demand")
frm_map.add_data(data = internal_demand, name="Internal/Internal Demand")
frm_map.add_data(data = internal_external_demand, name="Internal/External Demand")
frm_map.add_data(data = internal_taz, name="Internal TAZs")
frm_map.add_data(data = external_taz, name="External TAZs")
frm_map
frm_map.save_to_html(file_name=data_path+"/Data processing/Kepler maps/Demand/6pm_demand_count_widths_map.html")
'''

Number of Vehicles Analyzed at 6 PM (Internal): 3288
Number of Vehicles Analyzed at 6 PM (External): 1263
Number of Vehicles Analyzed at 6 PM (Total): 4551
User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


'\nfremont_map.save_to_html(file_name=data_path+"/Data processing/Kepler maps/Demand/6pm_demand_map.html")\nfrm_config = fremont_map.config\nfrm_map = KeplerGl(height=600, config=frm_config)\nfrm_map.add_data(data = external_demand, name="External/External Demand")\nfrm_map.add_data(data = internal_demand, name="Internal/Internal Demand")\nfrm_map.add_data(data = internal_external_demand, name="Internal/External Demand")\nfrm_map.add_data(data = internal_taz, name="Internal TAZs")\nfrm_map.add_data(data = external_taz, name="External TAZs")\nfrm_map\nfrm_map.save_to_html(file_name=data_path+"/Data processing/Kepler maps/Demand/6pm_demand_count_widths_map.html")\n'

## Open Street Map

In [39]:
import ogr

driver=ogr.GetDriverByName('OSM')
path_osm = data_path + "/Raw data (do not touch)/Network/Map/OSM/"


data = driver.Open(path_osm + 'map1111.osm')

for i in range(data.GetLayerCount()):
    print(data.GetLayerByIndex(i).GetName())
    
layer = data.GetLayer('points')

features=[x for x in layer]
print(len(features))

data_list=[]
i = 0
for feature in features:
    i = i + 1
    data=feature.ExportToJson(as_object=True)
    coords=data['geometry']['coordinates']
    shapely_geo=Point(coords[0],coords[1])
    name=data['properties']['name']
    highway=data['properties']['highway']
    print(highway)
    other_tags=data['properties']['other_tags']
    if other_tags and 'amenity' in other_tags:
        feat=[x for x in other_tags.split(',') if 'amenity' in x][0]
        amenity=feat[feat.rfind('>')+2:feat.rfind('"')]
    else:
        amenity=None
    data_list.append([name,highway,amenity,shapely_geo])
    if i > 20:
        break
# gdf=gpd.GeoDataFrame(data_list,columns=['Name','Highway','Amenity','geometry'],crs={'init': 'epsg:4326'}).to_crs(epsg=3310)

## External TAZs

In [38]:
from shapely.geometry import Point, LineString

path_taz = data_path + "/Manual-made dataset (do not touch)/Demand/OD demand/External TAZ/External Centroid zones/"
external_taz = to_gdf(path_taz + "/ExternalCentroidZones.shp")

fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = external_taz, name="External TAZs")
fremont_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'External TAZs': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 'columns': ['OBJECTID_1', 'FID_1…